In [23]:
import json
import pandas as pd
from json_ntv import Ntv, NtvList, NtvSingle
from datetime import date, time, datetime, timezone, timedelta

In [99]:
# pas les 'json'
types = pd.DataFrame({'ntv_type':  ['durationiso', 'uint64', 'float32', 'string', 'datetime', 'int32', 'int64', 
                                    'float64', 'array', 'boolean'], 
                      'name_type': [None, None, None, None, None, None, 'int64', 'float64', 'array', 'boolean'], 
                      'dtype':     ['timedelta64[ns]', 'UInt64', 'Float32', 'string', 'datetime64[ns]', 'Int32', 'Int64', 
                                    'Float64', 'object', 'boolean']})

def from_ntv(ntv_value, **kwargs):
    ''' convert ntv_value into the return object'''
    option = {'index':None, 'leng':None} | kwargs 
    ntv = Ntv.obj(ntv_value)
    ntv_name = ntv.name
    codes = len(ntv) == 2 and len(ntv[1]) > 1 # categorical
    if codes:
        cod = ntv[1].to_obj(simpleval=True)
        ntv = ntv[0]
    ntv_type = ntv.type_str if ntv.type_str != 'json' else ''
    pd_convert = ntv_type in types['ntv_type'].values or ntv_type == '' # pandas conversion

    if pd_convert:
        dtype = types.set_index('ntv_type').loc[ntv_type]['dtype'] if ntv_type != '' else None
        name_type = types.set_index('ntv_type').loc[ntv_type]['name_type'] if ntv_type != '' else ''
        pd_name = ntv_name + '::' + name_type if name_type else ntv_name
        pd_name = pd_name if pd_name else None
        list_val = ntv.obj_value()
        ntv_list = NtvList(list_val) if isinstance(list_val, list) else  NtvList([list_val])
        ntv_obj = ntv_list.to_obj() # add simpleval=True ?
    else:    
        dtype = 'object'
        name_type = ntv_type
        pd_name = ntv_name+'::'+name_type
        ntv_obj = ntv.to_obj(format='obj', simpleval=True, def_type=ntv_type)

    if codes:
        if pd_convert:
            dtype = types.set_index('ntv_type').loc[ntv_type]['dtype'] if ntv_type != '' else None
            print('dtype :', dtype)
            categories = pd.read_json(json.dumps(ntv_obj), dtype=dtype, typ='series')
            print('categories :', categories)
        else:
            #categories = pd.Series(ntv_obj, dtype=dtype)
            categories = pd.Series(ntv_obj, dtype='object')
        cat = pd.CategoricalDtype(categories=categories)
        cat.categories.astype()
        print('\ncat : ', cat.categories.dtype)
        data = pd.Categorical.from_codes(codes=cod, dtype=cat)
        print('data : ', data)
        return pd.Series(data, name=pd_name, index=option['index'] , dtype='category')
    
    data = ntv_obj * option['leng'] if len(ntv) == 1 and option['leng'] else ntv_obj
    if pd_convert:
        return pd.read_json(json.dumps(data), dtype=dtype, typ='series').rename(pd_name)
    return pd.Series(data, name=pd_name, dtype=dtype)

def ntv_type_val(name_type, sr):
    dtype = sr.dtype.name
    if not name_type:
        types_none = types.set_index('name_type').loc[None]
        if dtype in types_none.dtype.values:
            ntv_type = types_none.set_index('dtype').loc[dtype].ntv_type
        else:
            ntv_type = 'json'
        ntv_value = json.loads(sr.to_json(orient='records', date_format='iso', default_handler=str))
    else:
        ntv_type = name_type
        if dtype == 'object':
            ntv_value = sr.to_list()   
        else:
            ntv_value = json.loads(sr.to_json(orient='records', date_format='iso', default_handler=str))
    return (ntv_type, ntv_value)

def to_ntv(sr):
    ''' convert object into the NTV entity'''
    sr_name = sr.name if sr.name else ''
    ntv_name, name_type = Ntv.from_obj_name(sr_name)[:2]
    if sr.dtype.name == 'category':
        cdc = pd.Series(sr.cat.categories)    
        ntv_type, cat_value = ntv_type_val(name_type, cdc)
        ntv_value = [cat_value, NtvList(list(sr.cat.codes))]     
    else:
        ntv_type, ntv_value = ntv_type_val(name_type, sr)
    return NtvList(ntv_value, ntv_name, ntv_type).to_obj()


In [ ]:
def to_ntv(self):
    ''' convert object into the NTV entity (name, type, value)'''
    to_type = SeriesConnec.dtype_to_type
    dtype = self.dtype.name
    ntv_type = None
    ntv_name = self.name if self.name else ''
    if dtype in to_type and to_type[dtype] != 'json':
        ntv_type = to_type[dtype]
    if dtype in to_type:
        js = self.to_json(orient='records', date_format='iso', default_handler=str)
        ntv_value = json.loads(js)
    elif dtype == 'object':
        ntv_name, ntv_type, sep = Ntv.from_obj_name(ntv_name)
        ntv_value = self.to_list()
    elif dtype == 'category':
        ntv_name, ntv_type, sep = Ntv.from_obj_name(ntv_name)
        cdc = pd.Series(self.cat.categories)
        if cdc.dtype.name == 'object':
            ntv_value = [Ntv.obj(Ntv.obj(cdc).val),
                         NtvList(list(self.cat.codes))] 
        else:
            js = cdc.to_json(orient='records', date_format='iso', default_handler=str)
            ntv_value = [NtvList(json.loads(js), ntv_type=ntv_type),
                         NtvList(list(self.cat.codes))] 
        ntv_type = 'json'
    return (None, 'field', NtvList(ntv_value, ntv_name, ntv_type).to_obj())

In [85]:
import math
from shapely import geometry

srs = [# pas de ntv_type
       pd.Series([{'a': 2, 'e':4}, {'a': 3, 'e':5}, {'a': 4, 'e':6}]),  
       pd.Series([[1,2], [3,4], [5,6]]),  
       pd.Series([[1,2], [3,4], {'a': 3, 'e':5}]),  
       pd.Series([True, False, True]),
       pd.Series(['az', 'er', 'cd']),
       pd.Series([1,2,3]),
       pd.Series([1.1,2,3]),
       pd.Series([math.nan, math.nan]), # bug conversion json : datetime NaT
       
       pd.Series([1,2,3], dtype='Int64'),
       pd.Series([True, False, True], dtype='boolean'),
       pd.Series([1,2,3], dtype='Int64'), 
       
       # ajout d'un ntv_type dans le json uniquement
       pd.Series([pd.NaT, pd.NaT, pd.NaT]),
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns]'),
       pd.Series(pd.to_timedelta(['1D', '2D'])),
       pd.Series([1,2,3], dtype='Int32'), 
       pd.Series(['az', 'er', 'cd'], dtype='string'), 
       pd.Series([1,2,3], dtype='UInt64'), # not reversible
    
       # ajout d'un ntv_type dans le json et dans le nom
       pd.Series([1,2,3], name='::int64'),
       pd.Series([1,2,3], dtype='UInt64', name='::uint64'),   # name inutile
       pd.Series([1,2,3], dtype='Float64', name='::float64'), # force dtype dans la conversion json
       pd.Series([[1,2], [3,4], [5,6]], name='::array'),  
       pd.Series([None, None, None], name='::null'), 
       
       # type non connu de pandas
       pd.Series([date(2022, 1, 1), date(2022, 1, 2)], name='::date'),
       pd.Series([geometry.Point(1, 0), geometry.Point(1, 1), geometry.Point(1, 2)], name='::point'),
       pd.Series([1,2,3], dtype='object', name='::day'), # 

       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2), datetime(2022, 1, 3)], dtype='datetime64[ns, UTC]'), #1
]
for sr in srs:
    print(to_ntv(sr), from_ntv(to_ntv(sr)).equals(sr), from_ntv(to_ntv(sr)).name == sr.name)


[{'a': 2, 'e': 4}, {'a': 3, 'e': 5}, {'a': 4, 'e': 6}] True True
[[1, 2], [3, 4], [5, 6]] True True
[[1, 2], [3, 4], {'a': 3, 'e': 5}] True True
[True, False, True] True True
['az', 'er', 'cd'] True True
[1, 2, 3] True True
[1.1, 2.0, 3.0] True True
[None, None] False True
[1, 2, 3] True True
[True, False, True] True True
[1, 2, 3] True True
{'::datetime': [{':json': None}, {':json': None}, {':json': None}]} True True
{'::datetime': ['2022-01-01T00:00:00.000', '2022-01-02T00:00:00.000']} True True
{'::durationiso': ['P1DT0H0M0S', 'P2DT0H0M0S']} True True
{'::int32': [1, 2, 3]} True True
{'::string': ['az', 'er', 'cd']} True True
{'::uint64': [1, 2, 3]} True True
{'::int64': [1, 2, 3]} True True
{'::uint64': [1, 2, 3]} True False
{'::float64': [1.0, 2.0, 3.0]} True True
{'::array': [[1, 2], [3, 4], [5, 6]]} True True
{'::null': [{':json': None}, {':json': None}, {':json': None}]} True True
{'::date': ['2022-01-01', '2022-01-02']} True True
{'::point': [[1.0, 0.0], [1.0, 1.0], [1.0, 2.0]

C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\dtypes\cast.py:127: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)
C:\Opensource\anaconda3\envs\observ_v10_20230430\lib\site-packages\pandas\core\construction.py:809: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  subarr = construct_1d_object_array_from_listlike(arr)


In [86]:
for a in [{'test::int32': [1,2,3]},
          {'test': [1,2,3]},
          [1.0, 2.1, 3.0],
          ['er', 'et', 'ez'],
          [True, False, True],
          {'::boolean': [True, False, True]},
          {'::array': [{'a': 3, 'e':5}, {'a': 4, 'e':6}]},
          {'::string': ['er', 'et', 'ez']},
          {'test::float32': [1.0, 2.5, 3.0]},
          {'::int64': [1,2,3]},
          {'::datetime': ["2021-12-31T23:00:00.000","2022-01-01T23:00:00.000"] }]:
    print(to_ntv(from_ntv(a)) == a)

True
True
True
True
True
True
True
True
True
True
True


In [98]:
print(from_ntv({'test':[{'::float32': [1.0, 2.5, 3.0]}, [0,1,2,0,1,2]]},))

dtype : float32
categories : 0    1.0
1    2.5
2    3.0
dtype: float32

cat :  float64
data :  [1.0, 2.5, 3.0, 1.0, 2.5, 3.0]
Categories (3, float64): [1.0, 2.5, 3.0]
0    1.0
1    2.5
2    3.0
3    1.0
4    2.5
5    3.0
Name: test, dtype: category
Categories (3, float64): [1.0, 2.5, 3.0]


C:\Users\a lab in the Air\AppData\Local\Temp\ipykernel_11072\3538629589.py:44: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  print('data : ', data)
C:\Users\a lab in the Air\AppData\Local\Temp\ipykernel_11072\2085081995.py:1: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  print(from_ntv({'test':[{'::float32': [1.0, 2.5, 3.0]}, [0,1,2,0,1,2]]},))


In [87]:
for a in [{'test::int32': [[1,2,3], [0,1,2,0,1,2]]},
          {'test': [[1,2,3], [0,1,2,0,1,2]]},
          [[1.0, 2.1, 3.0], [0,1,2,0,1,2]],
          [['er', 'et', 'ez'], [0,1,2,0,1,2]],
          [[True, False], [0,1,0,1,0,1]],
          [{'::boolean': [True, False]}, [0,1,0,1,0,1]],
          #[{'::array': [{'a': 3, 'e':5}, {'a': 4, 'e':6}]}, [0,1,0,1]],
          [{'::string': ['er', 'et', 'ez']}, [0,1,2,0,1,2]],
          {'test':[{'::float32': [1.0, 2.5, 3.0]}, [0,1,2,0,1,2]]},
          [{'::int64': [1,2,3]}, [0,1,2,0,1,2]],
          [{'::datetime': ["2021-12-31T23:00:00.000","2022-01-01T23:00:00.000"] }, [0,1,0,1,0,1]]]:
    print(from_ntv(a))
    #print(to_ntv(from_ntv(a)))
    #print(to_ntv(from_ntv(a)) == a)
    #print(len(from_ntv(a)) == 6)

0    1
1    2
2    3
3    1
4    2
5    3
Name: test, dtype: category
Categories (3, int64): [1, 2, 3]
0    1
1    2
2    3
3    1
4    2
5    3
Name: test, dtype: category
Categories (3, int64): [1, 2, 3]
0    1.0
1    2.1
2    3.0
3    1.0
4    2.1
5    3.0
dtype: category
Categories (3, float64): [1.0, 2.1, 3.0]
0    er
1    et
2    ez
3    er
4    et
5    ez
dtype: category
Categories (3, object): ['er', 'et', 'ez']
0     True
1    False
2     True
3    False
4     True
5    False
dtype: category
Categories (2, bool): [True, False]
0     True
1    False
2     True
3    False
4     True
5    False
Name: ::boolean, dtype: category
Categories (2, bool): [True, False]
0    er
1    et
2    ez
3    er
4    et
5    ez
dtype: category
Categories (3, string): [er, et, ez]
0    1.0
1    2.5
2    3.0
3    1.0
4    2.5
5    3.0
Name: test, dtype: category
Categories (3, float64): [1.0, 2.5, 3.0]
0    1
1    2
2    3
3    1
4    2
5    3
Name: ::int64, dtype: category
Categories (3, int64): [1,

C:\Users\a lab in the Air\AppData\Local\Temp\ipykernel_11072\898656675.py:12: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  print(from_ntv(a))
C:\Users\a lab in the Air\AppData\Local\Temp\ipykernel_11072\898656675.py:12: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  print(from_ntv(a))


In [80]:
for a in [{'test::int32': [1]},
          {'test': 1},
          [2.1],
          'er',
          [True],
          {'::boolean': [True]},
          {':array': {'a': 3, 'e':5}},
          {'::string': ['er']},
          {'test:float32': 1.0},
          {'::int64': [1]},
          {':datetime': "2021-12-31T23:00:00.000" }]:
    #print(Ntv.obj(a).obj_value())
    print(len(from_ntv(a, leng=3)) == 3)

True
True
True
True
True
True
True
True
True
True
True


In [38]:
ntv = Ntv.obj({'test:float32': 1.0})

## mapping des types
Type json implicite (pas de ::type):
- json(number) - float64, int64
- json(string) - object(string)
- json(bool) - bool
- json(null) - NaT

Type pandas spécifique (::type non maintenu sur pandas):
- datetime - datetime64[ns]
- intxx, uintxx - Intxx, UIntxx
- floatxx - floatxx
- string - string

Type pandas explicite (::type maintenu sur pandas):
- float64 - float64
- int64 - Int64
- array - object
- boolean - bool
- null - NaT

Type  externe (::type maintenu sur pandas)

A voir
- datetime64[ns, <tz>]
- category
- period[<freq>]
- Sparse
- Interval

## conversion 
Type json:
- -> read_json(sans dtype)
- <- to_json

Type pandas spécifique ou explicite:
- -> read_json(avec dtype)
- <- to_json

Type externe:
- -> Ntv.to_obj(format='obj') -> Series
- <- Ntv.from_obj() <- Series

In [6]:
srs = [pd.Series([1,2,3], dtype='Int64'),
       pd.Series([True, False, True], dtype='boolean'),
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns]'),
       pd.Series([{'a': 3, 'e':5}, {'a': 4, 'e':6}]),  
       pd.Series([[3,4], [5,6]]),  
       pd.Series([[3,4], {'a': 3, 'e':5}]),  
       pd.Series([pd.NaT, pd.NaT, pd.NaT]),
       pd.Series([True, False, True]),
       pd.Series(['az', 'er', 'cd']),
       pd.Series([1,2,3]),
       pd.Series([1.1,2,3]),
       
       pd.Series([datetime(2022, 1, 1), datetime(2022, 1, 2)], dtype='datetime64[ns, UTC]'),
       pd.Series([1,2,3], dtype='Int32'),
       pd.Series([1,2,3], dtype='UInt64'),
       pd.Series([1,2,3], dtype='Float64'),
       pd.Series([1,2,3], dtype='object'),
       pd.Series([(3,4), (5,6)]),  
       pd.Series(['az', 'er', 'cd'], dtype='string'),
       pd.Series([None, None, None]),
       pd.Series([date(2022, 1, 1), date(2022, 1, 2)])
]
for sr in srs:
    print(sr.equals(pd.read_json(sr.to_json(orient='records', date_format='iso', default_handler=str), typ='series')))


True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
False
False
False
False
False


In [7]:
a =  {'test::int32': [1,2,3]}
#a = [{'a': 3, 'e':5}, {'a': 4, 'e':6}]
print(Ntv.obj(a).obj_value())
print([ntv.val for ntv in Ntv.obj(a).val])
print(NtvList(Ntv.obj(a).val).val)
print(from_ntv(a))
print(from_ntv(NtvList(Ntv.obj(a).val).val))
print(to_ntv(from_ntv(a)) == a)

[1, 2, 3]
[1, 2, 3]
["sT", "sT", "sT"]
0    1
1    2
2    3
Name: test, dtype: int32
0    1
1    2
2    3
dtype: int32
True


In [8]:
sr = pd.Series([1,2,3], dtype='float64')
print(sr)
print(sr.dtype.name)

0    1.0
1    2.0
2    3.0
dtype: float64
float64


In [9]:
ntv_value = {'::date': ['2022-01-01', '2022-01-02']}
ntv_val = Ntv.obj(ntv_value)
ntv_name = ntv_val.name
ntv_type = ntv_val.type_str if ntv_val.type_str != 'json' else ''
#ntv = NtvList(ntv.val)
ntv = NtvList(ntv_val.obj_value())
print(ntv)
obj = ntv_val.to_obj(format='obj', simpleval=True, def_type=ntv_type)
print(obj)
print(type(obj[0]))

["2022-01-01", "2022-01-02"]
[datetime.date(2022, 1, 1), datetime.date(2022, 1, 2)]
<class 'datetime.date'>


In [10]:
sr = pd.Series([{'a': 3, 'e':5}, {'a': 4, 'e':6}])
sr = pd.Series([1,2,3], dtype='Int64')
sr = pd.Series([1,2,3], dtype='Float64')
sr = pd.Series([date(2022, 1, 1), date(2022, 1, 2)], name='::date')
sr = pd.Series([None, None, None], name='::null')
sr = pd.Series([math.nan, math.nan])
sr = pd.Series([1,2,3], dtype='UInt64', name='::uint64')
       
ntv = to_ntv(sr)

ntv = Ntv.obj(ntv)
ntv_name = ntv.name
ntv_type = ntv.type_str if ntv.type_str != 'json' else ''
print(ntv_name, ntv_type)
#print(type(ntv_name), type(ntv_type))
sr_name = sr.name if sr.name else ''
ntv_name, name_type = Ntv.from_obj_name(sr_name)[:2]
dtype = sr.dtype.name
print(sr_name, ntv_name, name_type, dtype)
print(sr)
print(to_ntv(sr))
print(from_ntv(to_ntv(sr)))
print(sr.equals(from_ntv(to_ntv(sr))))
print(type(from_ntv(to_ntv(sr))[0]))

 uint64
::uint64 None uint64 uint64
0    1
1    2
2    3
Name: ::uint64, dtype: uint64
{'::uint64': [1, 2, 3]}
0    1
1    2
2    3
dtype: uint64
True
<class 'numpy.uint64'>


In [11]:
sr = pd.Series([date(2022, 1, 1), date(2022, 1, 2)], dtype='datetime64[ns, CET]')
print(sr)
sr2 = pd.Series([datetime(2022, 1, 1, tzinfo=timezone.utc), datetime(2022, 1, 2, tzinfo=timezone(timedelta(hours=2)))])
print(sr2)
print(type(sr2[1]))
print(sr.to_json(orient='records', date_format='iso', default_handler=str))
print(pd.read_json(sr.to_json(orient='records', date_format='iso', default_handler=str), typ='series'))
print(pd.read_json(sr.to_json(orient='records', default_handler=str), dtype='datetime64[ns, UTC]', typ='series'))


0   2022-01-01 00:00:00+01:00
1   2022-01-02 00:00:00+01:00
dtype: datetime64[ns, CET]
0    2022-01-01 00:00:00+00:00
1    2022-01-02 00:00:00+02:00
dtype: object
<class 'datetime.datetime'>
["2021-12-31T23:00:00.000","2022-01-01T23:00:00.000"]
0   2021-12-31 23:00:00
1   2022-01-01 23:00:00
dtype: datetime64[ns]
0   1970-01-01 00:27:20.991600+00:00
1   1970-01-01 00:27:21.078000+00:00
dtype: datetime64[ns, UTC]


In [12]:
sr = pd.Series([date(2022, 1, 1), date(2022, 1, 2)], dtype='datetime64[ns, CET]', name='test')
print(sr)
js = sr.to_json(orient='records', date_format='iso', default_handler=str)
sr2 = pd.read_json(js, dtype='datetime64[ns, UTC]', typ='series').rename('test2')
print(sr2)

0   2022-01-01 00:00:00+01:00
1   2022-01-02 00:00:00+01:00
Name: test, dtype: datetime64[ns, CET]
0   2021-12-31 23:00:00+00:00
1   2022-01-01 23:00:00+00:00
Name: test2, dtype: datetime64[ns, UTC]


In [13]:
a = pd.read_json('[false,true,null]', typ='series', dtype='Bool' )
print(type(a[0]))
print(type(a[2]))
print(a[0], '\n')
print(a)
print(a.to_json())

<class 'bool'>
<class 'NoneType'>
False 

0    False
1     True
2     None
dtype: object
{"0":false,"1":true,"2":null}


In [14]:
a = pd.read_json('["str1","str2",null]', typ='series')
print(type(a[0]))
print(type(a[2]))
print(a[0], '\n')
print(a)
print(a.to_json())

<class 'str'>
<class 'NoneType'>
str1 

0    str1
1    str2
2    None
dtype: object
{"0":"str1","1":"str2","2":null}
